In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [2]:
# !pip install progressbar2
from progressbar import progressbar

## Scraping

### Scraping Functions

In [3]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [4]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [5]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [6]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [7]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:43 Time:  0:00:43
100% (40 of 40) |########################| Elapsed Time: 0:00:38 Time:  0:00:38


In [8]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:41 Time:  0:00:41
100% (40 of 40) |########################| Elapsed Time: 0:00:39 Time:  0:00:39


In [9]:
nbatest.shape

(778, 3)

In [10]:
nfltest.shape

(903, 3)

##### These scrape_add functions add to already built csvs

In [11]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [13]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

In [14]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [15]:
train

,subreddit,title
t3_dva38v,nfl,Official /r/NFL Sidebar contest
t3_dvfdhm,nfl,Official Week 10 R/NFL Power Rankings
t3_dvrvvy,nfl,"[Falcons] YEAH, YOUNGHOE! 👉 @YoungHoeKoo has been named NFC Special Teams Player of the Week!"
t3_dvhdmh,nfl,"[Schefter] NFL clubs were informed today that a private workout will be held for Colin Kaepernick on Saturday in Atlanta. Session will include on-field work and an interview. All clubs are invited to attend, and video of both the workout and interview will be made available to clubs."
t3_dvrucj,nfl,[NFL] The AFC and NFC Offensive Players of the week are Lamar Jackson and Dalvin Cook
...,...,...
t3_du2fcc,nba,"Paul George on Kawhi Leonard - “Everything is true about his makeup. He works hard, he’s a great teammate, great locker room guy. He actually does talk and he’s a fun guy. The legend is true.”"
t3_dudqan,nba,Veteran Taj Gibson Joins Kevin Knox and Mitchell Robinson for Delta Road Bites
t3_du3gb5,nba,[Highlight] Gordon Hayward leaves the game after taking a hit from LaMarcus Aldridge on the screen
t3_duuarb,nba,What happens if a rookie plays entire games with no rest?


##### Tokenize (grab only word characters)

In [16]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [17]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [18]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [19]:
train['title'][0:5]

t3_dva38v                                                                                                                                                                                                                                                            official r nfl sidebar contest
t3_dvfdhm                                                                                                                                                                                                                                                     official week 10 r nfl power rankings
t3_dvrvvy                                                                                                                                                                                                     falcons yeah younghoe younghoekoo has been named nfc special teams player of the week
t3_dvhdmh    schefter nfl clubs were informed today that a private workout will be held for colin kaepernick on saturday in 

In [20]:
train['title'][0:5]

t3_dva38v                                                                                                                                                                                                                                                            official r nfl sidebar contest
t3_dvfdhm                                                                                                                                                                                                                                                     official week 10 r nfl power rankings
t3_dvrvvy                                                                                                                                                                                                     falcons yeah younghoe younghoekoo has been named nfc special teams player of the week
t3_dvhdmh    schefter nfl clubs were informed today that a private workout will be held for colin kaepernick on saturday in 

### Train test split and converting series to list of strings then to array

In [21]:
X = train[['title']]
y = train['subreddit']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [23]:
# baseline is

y.value_counts(normalize=True)

nfl    0.53718
nba    0.46282
Name: subreddit, dtype: float64

In [24]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [25]:
len(clean_train_data)

1260

In [26]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [27]:
len(clean_test_data)

421

### Count Vectorizer

In [28]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [29]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [30]:
# convert to array

train_data_features = train_data_features.toarray()

In [31]:
# check shapes

train_data_features.shape, test_data_features.shape

((1260, 5000), (421, 5000))

In [32]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [33]:
vocab[0:200]

['000',
 '10',
 '10 10',
 '10 14',
 '10 2019',
 '10 32',
 '10 assists',
 '10 game',
 '10 games',
 '10 games seahawks',
 '10 hawks',
 '10 hawks vs',
 '10 mnf',
 '10 nfl',
 '10 points',
 '10 teams',
 '10 waived',
 '10 weeks',
 '10 years',
 '100',
 '100 possessions',
 '100 yard',
 '100 yards',
 '1000',
 '101',
 '102',
 '104',
 '106',
 '107',
 '108',
 '108 87',
 '109',
 '11',
 '11 10',
 '11 10 hawks',
 '11 11',
 '11 12',
 '11 2019',
 '11 asts',
 '11 rebounds',
 '11 rebounds assists',
 '11 rebs',
 '11 shooting',
 '11 signed',
 '110',
 '111',
 '113',
 '114',
 '115',
 '116',
 '117',
 '119',
 '12',
 '12 2019',
 '12 22',
 '12 points',
 '12 team',
 '121',
 '123',
 '125',
 '12th',
 '13',
 '13 2019',
 '131',
 '136',
 '14',
 '14 shooting',
 '15',
 '15 pass',
 '15 shooting',
 '15 years',
 '150',
 '158',
 '158 rating',
 '16',
 '16 games',
 '16 points',
 '17',
 '176',
 '17th',
 '18',
 '19',
 '1950',
 '1983',
 '1984',
 '1985',
 '1986',
 '1990',
 '1st',
 '1st amp',
 '1st amp run',
 '20',
 '20 points',
 

## MODELING

### Logistic Regression

In [34]:
from sklearn.linear_model import LogisticRegression

In [35]:
# fit logistic regression model

lr = LogisticRegression(penalty='l2')

In [36]:
# shape check

train_data_features.shape, y_train.shape

((1260, 5000), (1260,))

In [37]:
lr.fit(train_data_features, y_train)

C:\Users\natha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
lr.score(train_data_features, y_train)

0.9984126984126984

In [39]:
lr.score(test_data_features, y_test)

0.9239904988123515

### Feature comparison

Creates a dataframe that matches features to coefficients

In [40]:
coef_list = lr.coef_.tolist()

In [41]:
coef_list = coef_list[0]

In [42]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [43]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
1866,nba,-2.298906
1605,lebron,-1.373281
1548,knicks,-1.165539
1511,kawhi,-1.094964
3307,raptors,-1.017577
...,...,...
4557,td,1.306260
4952,yards,1.314809
4857,week,1.416399
172,49ers,1.475019


### Let's throw out these unfair words and rerun

In [44]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [45]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1260, 5000), (421, 5000))

In [46]:
lr.fit(train_data_features, y_train)

C:\Users\natha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
lr.score(train_data_features, y_train)

0.9968253968253968

In [48]:
lr.score(test_data_features, y_test)

0.8907363420427553

In [49]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
1697,lebron,-1.365517
1625,knicks,-1.150627
1587,kawhi,-1.110473
4480,suns,-1.077362
1647,lakers,-1.030344
...,...,...
4947,yards,1.257305
2854,qb,1.279346
4524,td,1.327297
174,49ers,1.495135


### Decision Tree

In [50]:
from sklearn.tree import DecisionTreeClassifier

In [51]:
tree = DecisionTreeClassifier()

In [52]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [53]:
tree.score(train_data_features, y_train)

1.0

In [54]:
tree.score(test_data_features, y_test)

0.8242280285035629

### Random Forest

In [55]:
from sklearn.ensemble import RandomForestClassifier

In [56]:
forest = RandomForestClassifier(n_estimators = 100)

In [57]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [58]:
forest.score(train_data_features, y_train)

1.0

In [59]:
forest.score(test_data_features, y_test)

0.8717339667458432

###  Matrix on Logistic Regression

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
y_pred = lr.predict(test_data_features)

In [62]:
cm = confusion_matrix(y_test, y_pred)

In [63]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [64]:
cm_df

,predict_neg,predict_pos
actual_neg,181,14
actual_pos,32,194


## Checking where our model failed

In [65]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [66]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [67]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [68]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [69]:
mismatches

,y_actual,y_predicted,title
t3_du418t,nba,nfl,highlight robert timelord williams jumps out of the gym to catch the lob
t3_duhoax,nfl,nba,zrebiec jackson on whether that was his best run of his career it don t matter we scored and move on
t3_duhu9j,nba,nfl,agness pacers without turner bitadze lamb sumner and oladipo in orlando turner played some 4 on 4 this week but more time is needed to recover
t3_dui3ai,nfl,nba,highlight hoyer gets destroyered
t3_dujp31,nfl,nba,locker colts loyalty to the kicking goat vinatieri has them in a tough spot they ve fought their ass off this year but their margin for error is low and the poor kicking has them in a tough spot
t3_dujxvh,nfl,nba,d angelo dolphins are three games out of a wild card spot
t3_duldaf,nfl,nba,pft jarvis landry has noticed that opposing defenses are assigning their top cb to follow obj and landry regards that as disrespect
t3_dumnqx,nfl,nba,how do nfl players communicate during games when it s so loud with coaches with each other etc
t3_dumo5m,nfl,nba,the snf top gun segment was using the music from hot shots
t3_dusqmj,nba,nfl,kyle kuzma splits in his first 5 games 10 2 4 0 0 4 396 200 600


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [70]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=stopwords,
                            max_features=5000,
                            ngram_range=(1, 3))

In [71]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1260, 5000), (421, 5000))

In [72]:
lr.fit(train_data_features, y_train)

C:\Users\natha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
lr.score(train_data_features, y_train)

0.9920634920634921

In [74]:
lr.score(test_data_features, y_test)

0.9097387173396675

### Let's try on some other subreddits

In [75]:
train = pd.concat([politics_test, conservative_test])

In [76]:
X = train[['title']]
y = train['subreddit']

In [77]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [79]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [80]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [81]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [82]:
lr = LogisticRegression(penalty = 'l2')

In [83]:
train_data_features.shape, y_train.shape

((1310, 5000), (1310,))

In [84]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

C:\Users\natha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9778625954198473

In [85]:
lr.score(test_data_features, y_test)

0.8100686498855835

In [86]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [87]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
709,don,-1.062196
55,abc,-1.057494
113,alabama,-1.002882
250,believe,-0.955713
1029,haley trump,-0.904861
...,...,...
1163,impeachment hearings,1.095762
596,deal,1.098692
1159,impeachment,1.122041
4712,trump,1.397442


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [88]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [89]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [90]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [91]:
import requests

In [92]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [93]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [94]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [95]:
colors['olive']

(110, 117, 14)

In [96]:
colors['red']

(229, 0, 0)

In [97]:
colors['black']

(0, 0, 0)

In [98]:
colors['baby shit brown']

(173, 144, 13)

In [99]:
colors.keys()

dict_keys(['cloudy blue', 'dark pastel green', 'dust', 'electric lime', 'fresh green', 'light eggplant', 'nasty green', 'really light blue', 'tea', 'warm purple', 'yellowish tan', 'cement', 'dark grass green', 'dusty teal', 'grey teal', 'macaroni and cheese', 'pinkish tan', 'spruce', 'strong blue', 'toxic green', 'windows blue', 'blue blue', 'blue with a hint of purple', 'booger', 'bright sea green', 'dark green blue', 'deep turquoise', 'green teal', 'strong pink', 'bland', 'deep aqua', 'lavender pink', 'light moss green', 'light seafoam green', 'olive yellow', 'pig pink', 'deep lilac', 'desert', 'dusty lavender', 'purpley grey', 'purply', 'candy pink', 'light pastel green', 'boring green', 'kiwi green', 'light grey green', 'orange pink', 'tea green', 'very light brown', 'egg shell', 'eggplant purple', 'powder pink', 'reddish grey', 'baby shit brown', 'liliac', 'stormy blue', 'ugly brown', 'custard', 'darkish pink', 'deep brown', 'greenish beige', 'manilla', 'off blue', 'battleship gre

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [100]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1], [5, 2])

5.0990195135927845

The `subtractv` function subtracts one vector from another:

In [101]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [102]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [103]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [104]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

True

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [105]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [106]:
closest(colors, colors['white'])

['white',
 'pale grey',
 'very light pink',
 'off white',
 'ice blue',
 'very pale blue',
 'ice',
 'very light blue',
 'really light blue',
 'eggshell']

... or the ten colors closest to (150, 60, 150):

In [107]:
closest(colors, [255, 60, 150])

['barbie pink',
 'warm pink',
 'medium pink',
 'rosy pink',
 'darkish pink',
 'bubble gum pink',
 'strong pink',
 'electric pink',
 'bubblegum',
 'hot pink']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [108]:
closest(colors, subtractv(colors['green'], colors['purple']))

['true green',
 'racing green',
 'bottle green',
 'deep green',
 'darkgreen',
 'forest',
 'emerald green',
 'dark green',
 'vibrant green',
 'british racing green']

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [109]:
closest(colors, addv(colors['blue'], colors['green']))

['bright turquoise',
 'bright light blue',
 'bright aqua',
 'cyan',
 'neon blue',
 'aqua blue',
 'bright cyan',
 'bright sky blue',
 'aqua',
 'bright teal']

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [110]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

['medium grey',
 'purple grey',
 'steel grey',
 'battleship grey',
 'grey purple',
 'purplish grey',
 'greyish purple',
 'steel',
 'warm grey',
 'green grey']

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [111]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['orange'], colors['red'])
closest(colors, addv(pink_to_red, colors['blue']))

['azure',
 'turquoise blue',
 'bright sky blue',
 'aqua blue',
 'neon blue',
 'water blue',
 'topaz',
 'cerulean',
 'aqua',
 'dark sky blue']

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [112]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

['true green',
 'dark grass green',
 'grassy green',
 'racing green',
 'forest',
 'bottle green',
 'dark olive green',
 'darkgreen',
 'forrest green',
 'grass green']

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [113]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

Roses are red, violets are blue
Roses are orangish red, violets are rich blue
Roses are tomato red, violets are ultramarine blue
Roses are reddish orange, violets are royal blue
Roses are red orange, violets are indigo blue
Roses are orange red, violets are blue with a hint of purple
Roses are blood orange, violets are purple/blue
Roses are vermillion, violets are burple
Roses are red orange, violets are purple/blue
Roses are orangey red, violets are blurple
Roses are orangered, violets are iris
Roses are orange red, violets are bluey purple
Roses are red orange, violets are dark periwinkle
Roses are orangered, violets are iris


### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [114]:
import spacy
nlp = spacy.load('en_core_web_sm')

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [115]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

[147.44839067702551, 113.65371809100999, 100.13540510543841]


Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [116]:
closest(colors, avg_color)

['reddish grey',
 'brownish grey',
 'brownish',
 'brown grey',
 'mocha',
 'grey brown',
 'puce',
 'dull brown',
 'pinkish brown',
 'dark taupe']

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [117]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

['sickly yellow',
 'piss yellow',
 'puke yellow',
 'vomit yellow',
 'dirty yellow',
 'mustard yellow',
 'dark yellow',
 'olive yellow',
 'macaroni and cheese',
 'pea']

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [118]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

mauve trousers
dusty rose trousers
dusky rose trousers
brownish pink trousers
old pink trousers
reddish grey trousers
dirty pink trousers
old rose trousers
light plum trousers
ugly pink trousers


That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [119]:
dracula[560]

hour

In [120]:
_.vector

array([ 0.5709872 , -0.6691947 , -1.1217625 , -5.528201  ,  1.3921497 ,
        4.9203725 , -0.56104887,  1.6897587 , -0.08120264, -2.637401  ,
        1.4427888 ,  0.6387766 ,  1.5736012 , -1.7319528 , -0.15907216,
       -1.4701918 ,  0.2048592 , -3.7989602 , -1.7334095 , -1.3146017 ,
        2.1964169 , -1.0994648 ,  2.5789955 ,  1.7571774 ,  1.4141188 ,
       -0.09545761, -4.4905334 ,  6.025118  ,  1.2233729 ,  1.8006239 ,
       -1.5193732 , -1.9043732 ,  2.0958662 , -2.0175858 ,  0.62754625,
        0.2538525 , -1.1772894 ,  1.5775251 , -2.2222161 , -0.31077972,
        1.750873  ,  6.203821  , -0.32794297, -0.47397357,  0.6908189 ,
       -0.89367414,  1.5397241 , -2.1254988 ,  2.165449  , -0.6182877 ,
       -0.87687325, -2.6341186 , -4.1461005 , -1.5638533 , -3.503604  ,
        0.9974545 , -2.0495496 ,  3.884788  ,  3.3099852 , -1.9785341 ,
        2.658121  , -0.27347875,  1.8895068 , -0.6714524 ,  0.9160788 ,
        1.1273175 ,  1.059118  , -0.92695475,  1.8675635 ,  0.40

In [121]:
# previously we've used the _sm model, which doesn't include all vectors.
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
import en_core_web_lg
nlp = en_core_web_lg.load()

ModuleNotFoundError: No module named 'en_core_web_lg'

In [ ]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [ ]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [ ]:
nlp.vocab['alligator'].vector

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [ ]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

The cell below defines a function `cosine()`, which returns the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) of two vectors. Cosine similarity is another way of determining how similar two vectors are, which is more suited to high-dimensional spaces. [See the Encyclopedia of Distances for more information and even more ways of determining vector similarity.](http://www.uco.es/users/ma1fegan/Comunes/asignaturas/vision/Encyclopedia-of-distances-2009.pdf)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [ ]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [ ]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), vec(x))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [ ]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [ ]:
# halfway between day and night
spacy_closest(tokens, meanv([vec("day"), vec("night")]))

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [ ]:
spacy_closest(tokens, vec("wine"))

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [ ]:
spacy_closest(tokens, subtractv(vec("wine"), vec("alcohol")))

The closest words to "water":

In [ ]:
spacy_closest(tokens, vec("water"))

But if you add "frozen" to "water," you get "ice":

In [ ]:
spacy_closest(tokens, addv(vec("water"), vec("frozen")))

You can even do analogies! For example, the words most similar to "grass":

In [ ]:
spacy_closest(tokens, vec("grass"))

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [ ]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, addv(blue_to_sky, vec("grass")))

## Sentence similarity

To get the vector for a sentence, we simply average its component vectors, like so:

In [ ]:
def sentvec(s):
    sent = nlp(s)
    return meanv([w.vector for w in sent])

Let's find the sentence in our text file that is closest in "meaning" to an arbitrary input sentence. First, we'll get the list of sentences:

In [ ]:
sentences = list(doc.sents)

The following function takes a list of sentences from a spaCy parse and compares them to an input sentence, sorting them by cosine similarity.

In [ ]:
def spacy_closest_sent(space, input_str, n=10):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine_similarity(np.array(sentvec(str(x))).reshape(1, -1), np.array(input_vec).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Here are the sentences in *Dracula* closest in meaning to "My favorite food is strawberry ice cream." (Extra linebreaks are present because we didn't strip them out when we originally read in the source text.)

In [ ]:
for sent in spacy_closest_sent(sentences, "My favorite food is strawberry ice cream."):
    print(sent.text)
    print("---")

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.